<a href="https://colab.research.google.com/github/mshilpaa/ethereum-data-analysis/blob/master/Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Outgoing transactions:**
```
create table `civic-bruin-270815.ethereum.out_transaction` as (SELECT from_address as address, count(*) as no_of_out_transactions, sum(value) as tot_ether_sent,'OUT' as tax_type FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE from_address in (select address from `bigquery-public-data.crypto_ethereum.balances` LIMIT 400000) group by from_address) 
```

**Incoming transactions**
```
create table `civic-bruin-270815.ethereum.in_transaction` as (SELECT to_address as address, count(*) as no_of_in_transactions, sum(value) as tot_ether_recieved,'IN' as tax_type FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE to_address in (select address from `civic-bruin-270815.ethereum.in_transaction` LIMIT 2290371) group by to_address) 
```

**Join the two tables**
```
create table `civic-bruin-270815.ethereum.in_out_transaction` as (SELECT out_txn.address, out_txn.no_of_out_transactions, out_txn.tot_ether_sent, in_txn.no_of_in_transactions, in_txn.tot_ether_recieved from `civic-bruin-270815.ethereum.in_transaction` as in_txn inner join `civic-bruin-270815.ethereum.out_transaction` as out_txn on in_txn.address = out_txn.address) 
```

**Balance**
```
SELECT address, eth_balance FROM `bigquery-public-data.crypto_ethereum.balances` where address in (SELECT address FROM ``civic-bruin-270815.ethereum.in_out`)
```

**Tokens recieved**
```
SELECT to_address as address, count(distinct token_address) as tot_distinct_tokens_recieved, count(*) as tot_tokens_recieved, sum(safe_cast(value as FLOAT64)) as tot_token_value_recieved FROM `bigquery-public-data.crypto_ethereum.token_transfers` WHERE to_address in ( select address from `civic-bruin-270815.ethereum.in_out_transaction`) group by to_address
```

**Tokens sent**
```
SELECT from_address as address, count(distinct token_address) as tot_distinct_tokens_sent, count(*) as tot_tokens_sent, sum(safe_cast(value as FLOAT64)) as tot_token_value_sent FROM `bigquery-public-data.crypto_ethereum.token_transfers` WHERE from_address in ( select address from `civic-bruin-270815.ethereum.in_out_transaction`) group by from_address 
```

# **Sushmitha's Query**

**Time between incoming Transaction**



```
create table `psychic-timing-271718.ethereum.time_between_income_transaction` as (SELECT to_address as address, TIMESTAMP_DIFF(max(block_timestamp), min(block_timestamp),second)  as no_of_in_transactions, count(block_timestamp) as total_count, FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE to_address in (select address from `psychic-timing-271718.ethereum.in_out_transaction`) group by to_address)
```



**Time between outgoing Transaction**



```
create table `psychic-timing-271718.ethereum.time_between_out_transaction` as (SELECT from_address as address, TIMESTAMP_DIFF(max(block_timestamp), min(block_timestamp),second)  as no_of_out_transactions, count(block_timestamp) as total_out_count, FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE from_address in (select address from `psychic-timing-271718.ethereum.in_out_transaction`) group by from_address)
```



**Active Months**


```
with traces_clean as (
    select * from `bigquery-public-data.crypto_ethereum.transactions` 
    
), tx_received as (
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        to_address,
        count(*) as number_tx_received
    from traces_clean
    where to_address in (select address from psychic-timing-271718.ethereum.in_out_transaction)
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), to_address
    
), tx_sent as (
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        from_address,
        count(*) as number_tx_sent
    from traces_clean
    where from_address in (select address from psychic-timing-271718.ethereum.in_out_transaction)
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), from_address
    
), monthly_tx as (
    select 
        CASE  
          WHEN tx_sent.from_address IS NOT NULL THEN tx_sent.from_address
          WHEN tx_received.to_address IS NOT NULL THEN tx_received.to_address
        END AS address,
        CASE  
          WHEN tx_sent.month IS NOT NULL THEN tx_sent.month
          WHEN tx_received.month IS NOT NULL THEN tx_received.month
        END AS month,
        ifnull(number_tx_sent,0) as number_tx_sent, 
        ifnull(number_tx_received,0) as number_tx_received 
    from tx_sent full join tx_received 
        on (tx_sent.from_address = tx_received.to_address and tx_sent.month = tx_received.month)
)

select address, countif(number_tx_sent > 0 or number_tx_received > 0) as active_months from monthly_tx group by address ;
```



**Mothly_Outgoing_Transactions**



```
with traces_clean as (
    select * from `bigquery-public-data.crypto_ethereum.transactions` 
    
)
select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        from_address,
        count(*) as number_tx_sent
    from traces_clean
    where from_address in (select address from psychic-timing-271718.ethereum.in_out_transaction)
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), from_address
```

**After executing the above query, I have saved that as txn_sent table. Finally I have queried like below to get monthly_out_txn**


```
select from_address as address, sum(number_tx_sent)/count(month) as monthly_out_txn from psychic-timing-271718.ethereum.txn_sent group by from_address
```




**Monthly Incoming Transaations**



```
with traces_clean as (
    select * from `bigquery-public-data.crypto_ethereum.transactions` 
    
)
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        to_address,
        count(*) as number_tx_received
    from traces_clean
    where to_address in (select address from psychic-timing-271718.ethereum.in_out_transaction)
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), to_address
```

**This I saved as txn_recived and queried as below to get monthly_in_transaction**

```
select to_address as address, sum(number_tx_received)/count(month) as monthly_in_txn from psychic-timing-271718.ethereum.txn_recived group by to_address
```

